# Cleaning the Dataset

## Setup

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import json
import os.path
import string
from scripts.scrape import get_json, get_metadata

In [2]:
data_folder = os.path.join('..', 'data')

dirty_filename = os.path.join(data_folder, 'berlin_marathon_times_dirty.csv')
clean_filename = os.path.join(data_folder, 'berlin_marathon_times.csv')

dirty_filename

'../data/berlin_marathon_times_dirty.csv'

In [3]:
df = pd.read_csv(dirty_filename).sort_values(['year', 'id']).reset_index(drop=True)
df.head()

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year
0,2,19431,2002,Ohnemueller,Bert Martin,NaN,GER,1959,M,M45,2758,04:23:26,04:23:48,2005
1,3,10609,29356,Klinginger,Kurt,smart running team,GER,1947,M,M55,344,03:53:01,03:56:43,2005
2,4,23031,2034,Eleftheriadis,Georgios,NaN,GRE,1962,M,M40,4470,04:38:47,04:42:41,2005
3,5,14104,2035,Scholz,Detlef,NaN,GER,1970,M,M35,2573,04:03:50,04:05:14,2005
4,6,30239,F105,Frischke,Lucie,LAV Tempelhof Berlin,GER,1932,W,W70,2,06:07:10,06:09:04,2005


## Completeness Check

In [4]:
participants_year = df.year.value_counts().sort_index()
participants_year

2005    30373
2006    30182
2007    32486
2008    35731
2009    35015
2010    34001
2011    32914
2012    34270
2013    36473
2014    28945
2015    36753
2016    35991
Name: year, dtype: int64

In [5]:
participants_year_meta = participants_year.copy()

for year in participants_year.index:
    with open('../data/{0}-1.json'.format(year)) as f:
        j = json.load(f)
        m = get_metadata(j)
        participants_year_meta[year] = m.n_rows
        
participants_year_meta

2005    30373
2006    30182
2007    32486
2008    35731
2009    35015
2010    34001
2011    32914
2012    34270
2013    36473
2014    28945
2015    36753
2016    35991
Name: year, dtype: int64

In [6]:
len(participants_year[participants_year != participants_year_meta])

0

## Overview

In [7]:
len(df)

403134

In [8]:
df.dtypes

id                  int64
place               int64
bib                object
surname            object
forename           object
team               object
nationality        object
yob                 int64
sex                object
age_class          object
age_class_place     int64
net_time           object
clock_time         object
year                int64
dtype: object

In [9]:
df.describe()

,id,place,yob,age_class_place,year
count,403134.000000,403134.000000,403134.000000,403134.000000,403134.000000
mean,26240.607041,16899.358593,1967.243013,1596.684058,2010.619734
std,25441.663187,9861.602089,47.518969,1321.932040,3.426661
min,2.000000,1.000000,0.000000,0.000000,2005.000000
25%,10150.000000,8402.000000,1962.000000,517.000000,2008.000000
50%,20216.000000,16804.000000,1968.000000,1208.000000,2011.000000
75%,31350.000000,25206.000000,1976.000000,2456.000000,2014.000000
max,141120.000000,36768.000000,1998.000000,6101.000000,2016.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403134 entries, 0 to 403133
Data columns (total 14 columns):
id                 403134 non-null int64
place              403134 non-null int64
bib                403127 non-null object
surname            403096 non-null object
forename           403089 non-null object
team               181573 non-null object
nationality        403120 non-null object
yob                403134 non-null int64
sex                403132 non-null object
age_class          403105 non-null object
age_class_place    403134 non-null int64
net_time           403134 non-null object
clock_time         403134 non-null object
year               403134 non-null int64
dtypes: int64(5), object(9)
memory usage: 43.1+ MB


# Unique IDs

In [11]:
id_counts = df.groupby('id').year.count()
id_problems = id_counts[id_counts > 1]
len(id_problems)

58616

In [12]:
id_problems.describe()

count    58616.000000
mean         6.602378
std          3.688039
min          2.000000
25%          2.000000
50%          8.000000
75%         10.000000
max         12.000000
Name: year, dtype: float64

In [13]:
ids = df.groupby(['id', 'year']).place.count()
ids_problems = ids[ids > 1]
len(ids_problems)

0

## Place

In [14]:
len(df[df.place.isnull()])

0

In [15]:
df.place.describe()

count    403134.000000
mean      16899.358593
std        9861.602089
min           1.000000
25%        8402.000000
50%       16804.000000
75%       25206.000000
max       36768.000000
Name: place, dtype: float64

## Times

In [16]:
len(df[df.net_time.isnull()])

0

In [17]:
len(df[df.clock_time.isnull()])

0

In [18]:
df.net_time = pd.to_timedelta(df.net_time)
df.clock_time = pd.to_timedelta(df.clock_time)
df.net_time.describe()

count                    403134
mean     0 days 04:09:00.199402
std      0 days 00:42:32.086175
min             0 days 00:00:00
25%             0 days 03:39:12
50%             0 days 04:04:23
75%             0 days 04:35:43
max             0 days 08:41:33
Name: net_time, dtype: object

In [19]:
df.clock_time.describe()

count                    403134
mean     0 days 04:19:57.418917
std      0 days 00:48:21.491980
min             0 days 00:00:00
25%             0 days 03:45:24
50%             0 days 04:16:00
75%      0 days 04:51:08.750000
max             0 days 11:01:11
Name: clock_time, dtype: object

A time of zero seconds doesn't make sense so we replace it with null.

In [20]:
zero = pd.Timedelta(0)
len(df[df.net_time == zero])

2

In [21]:
df.net_time = df.net_time.replace(zero, np.nan)
len(df[df.net_time == zero])

0

In [22]:
len(df[df.clock_time == zero])

6

In [23]:
df.clock_time = df.clock_time.replace(zero, np.nan)
len(df[df.clock_time == zero])

0

In [24]:
times = df[df.net_time > df.clock_time]
times

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year
58489,41853,22138,F4217,Vogel,Esther,NaN,DEU,1973,W,W30,524,04:41:51,04:26:20,2006
312161,11182,2673,13549,Sowisnki,Fabrice,SAINT VENANT ATHLETISME,FRA,1974,M,40,595,03:13:55,02:58:59,2014


In [25]:
ev = df.loc[58489]
fs = df.loc[312161]

In [26]:
df[(df.year == ev.year) & (ev.place - 4 < df.place) & (df.place < ev.place + 5)].sort_values('place')

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year
38966,9778,22135,12254,von Blume,Marc,SV Blitzenreute,DEU,1962,M,M40,4357,04:41:50,04:43:48,2006
42175,13258,22136,8523,Kolke,Reinhard,NaN,DEU,1968,M,M35,3544,04:41:50,04:43:41,2006
52108,26030,22137,26910,Wojczewski,Axel-Rainer,NaN,DEU,1949,M,M55,936,04:41:51,04:46:29,2006
46026,17899,22138,F3482,Lorch,Renate,NaN,DEU,1960,W,W45,579,04:41:51,04:47:42,2006
58489,41853,22138,F4217,Vogel,Esther,NaN,DEU,1973,W,W30,524,04:41:51,04:26:20,2006
43553,14787,22140,16772,Kaspar,Werner,NaN,DEU,1953,M,M50,1985,04:41:52,04:45:27,2006
59977,43398,22141,13745,Ruholl,Steve,NaN,DEU,1979,M,MH,1817,04:41:52,04:45:31,2006
53329,28360,22142,29253,Fortune,Jean Pierre,NaN,FRA,1955,M,M50,1986,04:41:52,04:46:23,2006


In [27]:
df.set_value(ev.name, 'clock_time', ev.clock_time + pd.Timedelta('20 min'))
df.loc[ev.name]

id                           41853
place                        22138
bib                          F4217
surname                      Vogel
forename                    Esther
team                           NaN
nationality                    DEU
yob                           1973
sex                              W
age_class                      W30
age_class_place                524
net_time           0 days 04:41:51
clock_time         0 days 04:46:20
year                          2006
Name: 58489, dtype: object

In [28]:
df[(df.year == fs.year) & (fs.place - 4 < df.place) & (df.place < fs.place + 5)].sort_values('place')

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year
305127,3965,2670,6318,Borck,Jan,Der Kleine Muck,DEU,1964,M,50,200,03:13:55,03:14:12,2014
308016,6920,2671,26033,Schuller,Eelco,PAC,NLD,1986,M,H,264,03:13:55,03:15:41,2014
329255,28776,2672,6185,Boge,Georg,NaN,DEU,1980,M,30,464,03:13:55,03:18:29,2014
312161,11182,2673,13549,Sowisnki,Fabrice,SAINT VENANT ATHLETISME,FRA,1974,M,40,595,03:13:55,02:58:59,2014
306101,4960,2674,F597,Aljancic,Natasa,AK DOMALE,SVN,1969,W,45,13,03:13:56,03:15:45,2014
304271,3080,2675,7519,Chambers,Dion,NaN,GBR,1989,M,H,265,03:13:56,03:14:53,2014
306792,5671,2676,7933,Colsman,Philipp,NaN,DEU,1967,M,45,442,03:13:57,03:16:41,2014
327244,26721,2677,13784,Hinrichsen,Thomas,HRLK / BLL,DNK,1975,M,35,517,03:13:58,03:14:12,2014


In [29]:
df.set_value(fs.name, 'clock_time', ev.clock_time + pd.Timedelta('20 min'))
df.loc[fs.name]

id                                   11182
place                                 2673
bib                                  13549
surname                           Sowisnki
forename                           Fabrice
team               SAINT VENANT ATHLETISME
nationality                            FRA
yob                                   1974
sex                                      M
age_class                               40
age_class_place                        595
net_time                   0 days 03:13:55
clock_time                 0 days 04:46:20
year                                  2014
Name: 312161, dtype: object

In [30]:
len(df[df.net_time > df.clock_time])

0

## Sex

In [31]:
df.sex.describe()

count     403132
unique         2
top            M
freq      313715
Name: sex, dtype: object

In [32]:
df.sex.value_counts()

M    313715
W     89417
Name: sex, dtype: int64

In [33]:
no_sex = df[df.sex.isnull()]
len(no_sex)

2

In [34]:
no_sex

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year
391937,77124,27472,6956,Barris,Wolfram,NaN,NaN,1951,NaN,NaN,191,04:43:58,05:00:42,2016
392473,78774,26936,40651,Schnetzer,Wolfgang,NaN,NaN,1961,NaN,NaN,1509,04:41:32,05:15:56,2016


In [35]:
df.sex = df.sex.fillna('M').astype('category')
len(df[df.sex.isnull()])

0

## Year of Birth (YOB)

In [36]:
df.yob.describe()

count    403134.000000
mean       1967.243013
std          47.518969
min           0.000000
25%        1962.000000
50%        1968.000000
75%        1976.000000
max        1998.000000
Name: yob, dtype: float64

In [37]:
len(df[df.yob.isnull()])

0

In [38]:
df.yob = df.yob.replace(0, np.nan)
df.yob.describe()

count    402910.000000
mean       1968.336713
std          10.321517
min        1901.000000
25%        1962.000000
50%        1968.000000
75%        1976.000000
max        1998.000000
Name: yob, dtype: float64

It is unlikely to have a YOB of 1901.

In [39]:
df.yob.value_counts().sort_index().head(10)

1901.0    22
1902.0     3
1923.0     2
1925.0     2
1926.0     5
1927.0    12
1928.0    12
1929.0     8
1930.0    11
1931.0    22
Name: yob, dtype: int64

In [40]:
df['age'] = df.year - df.yob
df[df.yob <= 1925].sort_values(['yob', 'age'], ascending=[True, False])[['year', 'yob', 'age', 'age_class', 'net_time', 'place', 'age_class_place']]

,year,yob,age,age_class,net_time,place,age_class_place
393222,2016,1901.0,115.0,NaN,03:05:59,2126,0
168821,2010,1901.0,109.0,W,04:00:01,16646,0
171914,2010,1901.0,109.0,M,04:09:04,19394,0
172694,2010,1901.0,109.0,M,03:54:42,14287,0
179028,2010,1901.0,109.0,M,04:50:29,29460,0
179175,2010,1901.0,109.0,W,05:02:11,31018,0
180154,2010,1901.0,109.0,M,04:09:35,19565,0
183176,2010,1901.0,109.0,M,03:08:16,2071,0
189399,2010,1901.0,109.0,M,03:40:54,9474,0
190564,2010,1901.0,109.0,M,03:41:56,9751,0


In [41]:
df.yob = df.yob.replace(1901, np.nan).replace(1902, np.nan)
df['age'] = df.year - df.yob

Once we have cleaned up the age class, we can use this information to guess the YOB.

Are some listed as too young to participate?

In [42]:
df[df.age < 18]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
2291,2685,7246,5257,Koepke,Mathias,Mahlower SV,GER,1988.0,M,MJA,18,03:40:18,03:40:58,2005,17.0
4943,5704,26060,5466,Jacobsen,Eystein André,NaN,NOR,1988.0,M,MJA,97,04:55:34,04:59:23,2005,17.0
25172,31437,4107,29736,Petrausch,Janek,Schenker-TEAM,GER,1988.0,M,MJA,9,03:26:21,03:28:55,2005,17.0
61428,915,20059,2446,Musielak,Norman,NaN,DEU,1990.0,M,MJA,84,04:15:32,04:15:52,2007,17.0
61659,1154,30028,F1332,Scharpf,Maria,Scharpf-Family,DEU,1990.0,W,WJA,23,05:09:06,05:15:09,2007,17.0
77385,17477,32343,F6340,Garbe,Tabitha,NaN,DEU,1990.0,W,WJA,29,06:07:53,06:10:04,2007,17.0
139060,10843,24928,22567,Oppermann,Patrick,NaN,DEU,1995.0,M,MJA,79,04:35:04,04:42:02,2009,14.0
167596,4134,20039,F8916,Forsberg,Sofie,MTC,DNK,1993.0,W,WJA,7,04:11:04,04:33:41,2010,17.0


## Age Class

In [43]:
df.age_class.value_counts().sort_index()

30     25394
35     30177
40     38211
45     38433
50     28243
55     13845
60      6461
65      2477
70       971
75       255
80        37
H      20298
JA       420
M         25
M30    18301
M35    26373
M40    33912
M45    27643
M50    17945
M55     9114
M60     4376
M65     2058
M70      597
M75       82
M80       15
MH     16315
MJA      724
U20       95
W          8
W30     5474
W35     6598
W40     8675
W45     7093
W50     4033
W55     1618
W60      584
W65      251
W70       50
W75        3
W80        5
WH      5742
WJA      174
Name: age_class, dtype: int64

The age class groups are very inconsistent in their naming and many also contain information about the sex of the participant.  Before removing the sex from the age class, we can use it as a consistency check for the sex column

In [44]:
len(df[df.age_class.str.contains(r'[MW]', na=False)])

197788

In [45]:
age_class_sex = df[df.age_class.str.contains(r'[MW]', na=False)].age_class.str.get(0)
inconsistent_sex0 = df.loc[age_class_sex.index, 'sex'] != age_class_sex
inconsistent_sex = df.loc[inconsistent_sex0[inconsistent_sex0 == True].index, :]
len(inconsistent_sex)

34

In [46]:
inconsistent_sex

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
9201,10341,1532,F3923,Lohmann,Henrik,Sparta,DEN,1964.0,W,M40,353,03:06:49,03:07:37,2005,41.0
10108,11280,14344,F5717,Sanchez,Raul,NaN,SUI,1962.0,W,M40,3067,04:04:49,04:08:49,2005,43.0
19935,23704,10930,F4420,Kachlik,Stephan,NaN,GER,1963.0,W,M40,2410,03:54:04,03:56:47,2005,42.0
20122,23897,7434,F5646,Kirchhoff,Jochen,TSV Bollensen,GER,1957.0,W,M45,1085,03:41:07,03:44:33,2005,48.0
23362,28829,9129,F6506,Pape,Gerald,TriFrogs Gellersen,GER,1965.0,W,M40,2017,03:47:41,03:49:22,2005,40.0
26631,33336,21772,F8928,Tricoche,Elisabeth,Defi Horizon,FRA,1949.0,W,M55,840,04:33:09,04:37:12,2005,56.0
30198,37786,6827,32681,Filippo,Proietti,NaN,ITA,1967.0,M,W35,102,03:38:31,03:40:10,2005,38.0
30200,37789,20204,32699,Demetrio,Alberto,NaN,BRA,1965.0,M,W40,677,04:26:27,04:30:40,2005,40.0
34335,4614,2110,F3676,Aeby,Pascal Laurent,NaN,DEU,1966.0,W,M40,495,03:16:09,03:16:17,2006,40.0
44551,16028,106,F7055,Nordgaard,Haavard,Nike,NOR,1961.0,W,M45,3,02:38:57,02:39:04,2006,45.0


Judging by the names, the real sex is usually in the age class.

In [47]:
inconsistent_sex.sex = inconsistent_sex.age_class.str.get(0)
inconsistent_sex

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
9201,10341,1532,F3923,Lohmann,Henrik,Sparta,DEN,1964.0,M,M40,353,03:06:49,03:07:37,2005,41.0
10108,11280,14344,F5717,Sanchez,Raul,NaN,SUI,1962.0,M,M40,3067,04:04:49,04:08:49,2005,43.0
19935,23704,10930,F4420,Kachlik,Stephan,NaN,GER,1963.0,M,M40,2410,03:54:04,03:56:47,2005,42.0
20122,23897,7434,F5646,Kirchhoff,Jochen,TSV Bollensen,GER,1957.0,M,M45,1085,03:41:07,03:44:33,2005,48.0
23362,28829,9129,F6506,Pape,Gerald,TriFrogs Gellersen,GER,1965.0,M,M40,2017,03:47:41,03:49:22,2005,40.0
26631,33336,21772,F8928,Tricoche,Elisabeth,Defi Horizon,FRA,1949.0,M,M55,840,04:33:09,04:37:12,2005,56.0
30198,37786,6827,32681,Filippo,Proietti,NaN,ITA,1967.0,W,W35,102,03:38:31,03:40:10,2005,38.0
30200,37789,20204,32699,Demetrio,Alberto,NaN,BRA,1965.0,W,W40,677,04:26:27,04:30:40,2005,40.0
34335,4614,2110,F3676,Aeby,Pascal Laurent,NaN,DEU,1966.0,M,M40,495,03:16:09,03:16:17,2006,40.0
44551,16028,106,F7055,Nordgaard,Haavard,Nike,NOR,1961.0,M,M45,3,02:38:57,02:39:04,2006,45.0


In [48]:
inconsistent_sex.set_value(99806, 'sex', 'M')
inconsistent_sex.set_value(96128, 'sex', 'W')
inconsistent_sex.set_value(86445, 'sex', 'M')
inconsistent_sex.set_value(69640, 'sex', 'M')
inconsistent_sex.set_value(58571, 'sex', 'M')
inconsistent_sex.set_value(30200, 'sex', 'M')
inconsistent_sex.set_value(30198, 'sex', 'M')

df.loc[inconsistent_sex.index, 'sex'] = inconsistent_sex.sex
df.loc[inconsistent_sex.index, :]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
9201,10341,1532,F3923,Lohmann,Henrik,Sparta,DEN,1964.0,M,M40,353,03:06:49,03:07:37,2005,41.0
10108,11280,14344,F5717,Sanchez,Raul,NaN,SUI,1962.0,M,M40,3067,04:04:49,04:08:49,2005,43.0
19935,23704,10930,F4420,Kachlik,Stephan,NaN,GER,1963.0,M,M40,2410,03:54:04,03:56:47,2005,42.0
20122,23897,7434,F5646,Kirchhoff,Jochen,TSV Bollensen,GER,1957.0,M,M45,1085,03:41:07,03:44:33,2005,48.0
23362,28829,9129,F6506,Pape,Gerald,TriFrogs Gellersen,GER,1965.0,M,M40,2017,03:47:41,03:49:22,2005,40.0
26631,33336,21772,F8928,Tricoche,Elisabeth,Defi Horizon,FRA,1949.0,M,M55,840,04:33:09,04:37:12,2005,56.0
30198,37786,6827,32681,Filippo,Proietti,NaN,ITA,1967.0,M,W35,102,03:38:31,03:40:10,2005,38.0
30200,37789,20204,32699,Demetrio,Alberto,NaN,BRA,1965.0,M,W40,677,04:26:27,04:30:40,2005,40.0
34335,4614,2110,F3676,Aeby,Pascal Laurent,NaN,DEU,1966.0,M,M40,495,03:16:09,03:16:17,2006,40.0
44551,16028,106,F7055,Nordgaard,Haavard,Nike,NOR,1961.0,M,M45,3,02:38:57,02:39:04,2006,45.0


In [49]:
df.age_class = df.age_class.str.replace(r'[MW]?(.*)', r'\1')
df.age_class.value_counts().sort_index()

          33
30     49169
35     63148
40     80798
45     73169
50     50221
55     24577
60     11421
65      4786
70      1618
75       340
80        57
H      42355
JA      1318
U20       95
Name: age_class, dtype: int64

In [50]:
len(df[(df.age_class == "")])

33

In [51]:
df.age_class = df.age_class.replace('', np.nan)
len(df[df.age_class == ""])

0

In [52]:
df.groupby('age_class').age.aggregate(['min', 'max'])

,min,max
age_class,,
30,30.0,54.0
35,32.0,40.0
40,39.0,44.0
45,45.0,49.0
50,47.0,54.0
55,55.0,59.0
60,60.0,64.0
65,65.0,69.0
70,60.0,74.0


In [53]:
df[(df.age_class == '30') & (df.age >= 35)]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
122666,34733,4928,36826,Silva,Usiel,Rho Delta Trave,BRA,1954.0,M,30,677,03:23:32,03:23:38,2008,54.0


In [54]:
df.set_value(122666, 'age_class', '50')

df[(df.age_class == '35') & (df.age < 35)]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
124745,38362,16994,24442,Neuhaus,Svenja,NaN,DEU,1976.0,W,35,2725,03:59:50,04:14:42,2008,32.0


In [55]:
df.set_value(124745, 'age_class', '30')

df[(df.age_class == '50') & (df.age < 50)]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
101200,8907,2532,17349,Lameir,Christian,NaN,DEU,1959.0,M,50,155,03:10:18,03:10:26,2008,49.0
261808,32192,20225,13846,Hundertmark,Peter,NaN,DEU,1965.0,M,50,2125,04:12:00,04:22:19,2012,47.0


In [56]:
df.set_value(261808, 'age_class', '45')

df[(df.age_class == '70') & (df.age < 70)]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
144135,16097,28932,F174,Kiederle,Inger,Atletica,DNK,1942.0,W,70,3,04:52:07,05:07:35,2009,67.0
160277,32876,20390,37361,Thiel,Peter,NaN,DEU,1949.0,M,70,19,04:19:01,04:30:43,2009,60.0


In [57]:
df.set_value(144135, 'age_class', '65')
df.set_value(160277, 'age_class', '60')

df[(df.age_class == 'H') & (df.age >= 30)]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
81816,22121,41,84,Güssow,Christian,Hamburger SV,DEU,1976.0,M,H,17,02:27:05,02:27:08,2007,31.0
147825,19946,14485,23852,Pranzner,Andreas,Berliner Feuerwehr,DEU,1961.0,M,H,1415,04:00:00,04:19:35,2009,48.0


In [58]:
df.set_value(81816, 'age_class', '30')
df.set_value(147825, 'age_class', '45')

def upper(match):
    if match.group(2) == '0':
        d = '4'
    else:
        d = '9'
    return '{0}-{1}{2}'.format(match.group(0), match.group(1), d)

df.age_class = df.age_class.str.replace(r'^(JA|U20)$', '0-19').str.replace('^H$', '20-29').str.replace(r'^8.*', '80+').str.replace(r'^(\d)(\d)$', upper)
df.groupby('age_class').age.aggregate(['min', 'max'])

,min,max
age_class,,
0-19,14.0,19.0
20-29,20.0,29.0
30-34,30.0,34.0
35-39,35.0,40.0
40-44,39.0,44.0
45-49,45.0,49.0
50-54,49.0,54.0
55-59,55.0,59.0
60-64,60.0,64.0


In [59]:
age_classes = pd.unique(df.age_class.dropna())
df.age_class = df.age_class.astype('category', categories=age_classes, ordered=True)

## Age Class Place

In [60]:
df.age_class_place.describe()

count    403134.000000
mean       1596.684058
std        1321.932040
min           0.000000
25%         517.000000
50%        1208.000000
75%        2456.000000
max        6101.000000
Name: age_class_place, dtype: float64

In [61]:
len(df[df.age_class_place.isnull()])

0

In [62]:
df.age_class_place = df.age_class_place.replace(0, np.nan)
yob_no_ac = df[(df.age_class_place.isnull()) & (~df.yob.isnull())]
yob_no_ac.age_class = (5 * (yob_no_ac.age // 5)).astype('int').astype('str').str.replace(r'^(\d)(\d)$', upper).astype('category', categories=age_classes)
yob_no_ac

/home/brian/.virtualenvs/generic/lib/python3.5/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
177069,13887,9534,NaN,Coll,Florian,Cours Forest,FRA,1983.0,M,NaN,NaN,03:41:07,03:52:11,2010,27.0
250216,20259,12554,3612,Adriaansen,Dave,NaN,NLD,1977.0,M,35-39,NaN,03:49:46,04:00:05,2012,35.0
256755,26984,6633,1600,Van Den Bergh,Dennis,NaN,DEU,1969.0,M,40-44,NaN,03:29:36,03:35:05,2012,43.0
354959,93204,3633,38826,Hofmann,Lars,Tv Großen Buseck,GER,1979.0,M,35-39,NaN,03:13:33,03:14:14,2015,36.0
357198,101622,20,100,Solomon,Tesfamariam,TVL Bern,ERI,1976.0,M,35-39,NaN,02:14:51,02:14:51,2015,39.0
367138,141090,36483,F12454,Stasch,Christa,NaN,GER,1956.0,W,55-59,NaN,06:10:32,06:46:46,2015,59.0
367139,141108,35431,38838,Wang,Hsi-Tsang,NaN,TPE,1976.0,M,35-39,NaN,05:36:47,06:16:10,2015,39.0
367140,141111,9428,38825,Heinrichs,Markus,SCW München Triathlon,GER,1980.0,M,35-39,NaN,03:36:51,03:56:07,2015,35.0
367141,141114,19328,38827,Huai,Qianjiang,NaN,CHN,1975.0,M,40-44,NaN,04:04:35,04:24:01,2015,40.0
367142,141120,18143,38828,Kraus,Martin,NaN,GER,1975.0,M,40-44,NaN,04:00:32,04:23:56,2015,40.0


In [63]:
df.loc[yob_no_ac.index, 'age_class'] = yob_no_ac.age_class

## Teams

In [64]:
df.team.describe()

count       181573
unique       68777
top       Terramia
freq          1908
Name: team, dtype: object

## Nationality

In [65]:
df.nationality.describe()

count     403120
unique       250
top          DEU
freq      155501
Name: nationality, dtype: object

In [66]:
len(df[df.nationality.isnull()])

14

[ISO](https://en.wikipedia.org/wiki/ISO_3166-1)
[FIFA](https://en.wikipedia.org/wiki/List_of_FIFA_country_codes)
[IOC](https://en.wikipedia.org/wiki/List_of_IOC_country_codes)

In [67]:
nat = pd.read_csv('../data/countries.csv')
nat.head()

,abbreviation,iso3166_1alpha_3,country
0,ABW,ABW,Aruba
1,AFG,AFG,Afghanistan
2,AGO,AGO,Angola
3,AHO,AHO,Netherlands Antilles
4,ALB,ALB,Albania


In [68]:
sorted(set(df.nationality.dropna()) - set(nat.abbreviation))

['RKS']

In [69]:
dfnat = pd.merge(df, nat, left_on='nationality', right_on='abbreviation', how='left').sort_values(['year', 'id']) #.reindex(df.index)
(df.index == dfnat.index).min()

True

In [70]:
df.nationality = dfnat.iso3166_1alpha_3
df.nationality.describe()

count     403117
unique       199
top          DEU
freq      192234
Name: nationality, dtype: object

## Names

In [71]:
df.forename.describe()

count     403089
unique     37436
top       Thomas
freq        7865
Name: forename, dtype: object

In [72]:
len(df[df.forename.isnull()])

45

In [73]:
df[(df.forename.isnull()) & (~df.surname.isnull())]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
60554,44151,15749,29772,"Chávez-Fernández Goyburu,Juan José",NaN,Regatas Lima,PER,1967.0,M,35-39,2771.0,04:15:47,04:18:00,2006,39.0
60901,359,31004,F1115,"Mackenzie,",NaN,NaN,NLD,1962.0,W,45-49,1028.0,05:21:45,05:29:38,2007,45.0
94924,2386,22570,F2721,"Elvers-Schreiber,",NaN,DAV Berlin,DEU,1951.0,W,55-59,86.0,04:17:47,04:22:01,2008,57.0
122234,34063,17214,F2727,"Emmanuelle Rochet-Blanc,",NaN,les chauffe la semelle,FRA,1968.0,W,40-44,433.0,04:00:22,04:06:18,2008,40.0
124378,37491,34879,37896,"Williams,",NaN,NaN,DEU,NaN,M,20-29,2921.0,05:38:09,05:44:09,2008,NaN
124379,37492,27883,37897,"Keith,",NaN,NaN,GBR,NaN,M,20-29,2452.0,04:37:34,04:41:15,2008,NaN
125273,39885,4790,37898,"Witfield,",NaN,NaN,GBR,NaN,M,20-29,503.0,03:22:56,03:24:39,2008,NaN
132211,3741,836,6092,Bosgoed,NaN,ARO'88,NLD,1966.0,M,40-44,183.0,02:57:05,02:57:29,2009,43.0
266361,1602,6254,10087,Gestin,NaN,CACL Courbevoie,FRA,1967.0,M,45-49,1086.0,03:27:16,03:30:36,2013,46.0


In [74]:
df.set_value(60554, 'forename', 'Juan José')
df.set_value(60554, 'surname', 'Chávez-Fernández Goyburu')

df.surname.describe()

count      403096
unique     127265
top       Nielsen
freq         1648
Name: surname, dtype: object

In [75]:
len(df[df.surname.isnull()])

38

In [76]:
df[(df.surname.isnull()) & (~df.forename.isnull())]

,id,place,bib,surname,forename,team,nationality,yob,sex,age_class,age_class_place,net_time,clock_time,year,age
30385,16,12730,32232,NaN,Benjamin,NaN,DEU,1982.0,M,20-29,1134.0,04:04:33,04:10:01,2006,24.0
148108,20236,223,26347,NaN,Christoph,LAV Asics Tübingen,DEU,1984.0,M,20-29,53.0,02:41:58,02:42:06,2009,25.0


In [77]:
df.surname = df.surname.str.strip(string.punctuation)
df.forename = df.forename.str.strip(string.punctuation)

## Bibs

In [78]:
df.bib.describe()

count     403127
unique     59339
top        F1024
freq          37
Name: bib, dtype: object

In [79]:
len(df[df.bib.isnull()])

7

## Save the Clean Data

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403134 entries, 0 to 403133
Data columns (total 15 columns):
id                 403134 non-null int64
place              403134 non-null int64
bib                403127 non-null object
surname            403096 non-null object
forename           403090 non-null object
team               181573 non-null object
nationality        403117 non-null object
yob                402885 non-null float64
sex                403134 non-null category
age_class          403078 non-null category
age_class_place    403071 non-null float64
net_time           403132 non-null timedelta64[ns]
clock_time         403128 non-null timedelta64[ns]
year               403134 non-null int64
age                402885 non-null float64
dtypes: category(2), float64(3), int64(3), object(5), timedelta64[ns](2)
memory usage: 40.8+ MB


In [81]:
df.net_time = df.net_time.astype(str).str.split().str[-1].str.rsplit('.').str[0]
df.clock_time = df.clock_time.astype(str).str.split().str[-1].str.rsplit('.').str[0]

df.sort_values(['year', 'id']).to_csv(clean_filename, index=True, index_label='idx')